This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

In [908]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [900]:
print(os.listdir("sd/LOG"))

['001.TXT', '002.TXT', '003.TXT', '004.TXT', '005.TXT', '006.TXT', '007.TXT', '008.TXT', '009.TXT', '010.TXT', '011.TXT', '012.TXT', '013.TXT', '014.TXT', '015.TXT', '016.TXT']


In [901]:
%fetchfile --binary "sd/LOG/016.TXT"


Fetched 212813=212813 bytes from sd/LOG/016.TXT.
Saving file to '016.TXT'

In [822]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [938]:
%sendtofile --source NewDataLogger_funcs.py


Sent 89 lines (3191 bytes) to NewDataLogger_funcs.py.


In [906]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py


Sent 3368 bytes in 113 chunks to OLED_driver.py.


In [494]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py
%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BlueFly_funcs.py


Sent 55 lines (2040 bytes) to SI7021_funcs.py.


In [933]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [927]:
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly

ConnectBluefly()
sdfile = ConnectSDcardFile()
print(sdfile)

I (10380) uart: ALREADY NULL


Opening file sd/LOG/018.TXT
None


In [939]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART
import sdcard, os, time
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from BlueFly_funcs import uart, SetupGPS, BFVmakesettings, ParseNMEA
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly, readlogSI7021, calchumidtemps, readlogbluefly
                       
#ConnectBluefly()
sdfile = ConnectSDcardFile()
print(sdfile)

from SI7021_funcs import setupSI7021, SI7021checkchip, SI7021printstatus, SI7021humiditytemp, SI7021humiditytempBin
setupSI7021(i2c)
SI7021checkchip()

prevflushstamp = 0
prevoledstamp = 0
while True:
    tstamp = time.ticks_ms()
    readlogSI7021()
    prs = readlogbluefly()
    if tstamp > prevflushstamp + 2000:
        sdfile.flush()
        prevflushstamp = tstamp
        
    if tstamp > prevoledstamp + 200:
        #v = calchumidtemps()[1]
        if prs > 0:
            addscrollgraph(prs-100000, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("prs",0,0,1)
            fbuff.text(str(prs),0,8,1)
        else:
            oledshowfattext(["mstamp", str(tstamp)])
        oledshow()
        prevoledstamp = tstamp
                


Sent 37 lines (1243 bytes) to main.py.


In [941]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f21434a63c8, open=True>(port='/dev/ttyUSB4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [372]:
%fetchfile --binary sd/LOG/001.TXT

Fetched 69930=69930 bytes from sd/LOG/001.TXT.
Saving file to '001.TXT'

In [940]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [377]:
import os
print(os.listdir("sd/LOG"))

['001.TXT', '002.TXT', '003.TXT']


In [ ]:
# Nickel wire setup code
import machine, time
x = machine.ADC(machine.Pin(36))
x.atten(x.ATTN_11DB)
x.width(x.WIDTH_12BIT)

fout = open("data4.txt", "w")
for i in range(6000):
    fout.write("%d,%d\n" % (time.ticks_ms(), x.read()))
    if (i%1000) == 0:
        print(i)
fout.close()


In [553]:
uart

In [737]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [453]:
print(list(os.ilistdir("sd/LOG")))

[('001.TXT', 32768, 0, 69930), ('002.TXT', 32768, 0, 30183), ('003.TXT', 32768, 0, 27947)]


In [746]:
print(1)

1


In [803]:
from BlueFly_funcs import uart, SetupGPS, BFVmakesettings
BFVmakesettings({ b"BOF":1, # Output frequency (factor of 20ms)
                  b"BHV":0, # height timeout
                  b"BHT":10000, # seconds to timeout
                  b"BVL":3  # volume (out of 1000)
                })
print(SetupGPS())

b'BVL' already good
b'BHV' already good
b'BOF' already good
b'BHT' already good
GPS already baud 57600
True


In [772]:
print(bd)

[b'$GPRMC', b'121512.600', b'A', b'5256.5498', b'N', b'00407.2869', b'W', b'0.00', b'326.11', b'080618', b'', b'', b'A*7C\r\n']


In [819]:
import time
for i in range(2000):
    lbd = uart.readline()
    if lbd:
        if lbd[:1] == b"$":
            print(ParseNMEA(lbd), end="")
    time.sleep_ms(1)
    

Qt002C2FC6u02BFFAC0y01E4B447x-025BB58a0076
Vt002C2FCDv0001d00761C
Vt002C2FDAv0001d006FE5
Qt002C305Cu02C017A8y01E4B447x-025BB58a0075
Vt002C3062v0001d006C52
Qt002C3123u02C01870y01E4B447x-025BB58a0075
b'$GPRMC,124903.600,A,5256.5575,N,00407.2792,W,0.00,344.36,080618,,,D*78\r\n'
31765575 -2472792 34436
NoneVt002C313Ev0001d008684
Qt002C31ECu02C01938y01E4B447x-025BB58a0075
Vt002C31F2v0001d0026F3
Qt002C32B3u02C01A00y01E4B447x-025BB58a0075
Vt002C32B9v0001d00237B
Qt002C337Au02C01AC8y01E4B447x-025BB58a0075
Vt002C3381v0001d003C04
Qt002C3442u02C01B90y01E4B447x-025BB58a0075
Vt002C3448v0000d0044B1
Qt002C3508u02C01C58y01E4B447x-025BB58a0075
b'$GPRMC,124904.600,A,5256.5575,N,00407.27'


Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
  File "<stdin>", line 16, in ParseNMEA
IndexError: list index out of range
